# CNP_func_validation_age

In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
from sklearn.svm import LinearSVR
# from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
from nilearn.input_data import NiftiMasker
import random
import matplotlib.pyplot as plt
import datetime

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Config
toy = False
run_all = False



#AWS cloud
# path = '/home/ubuntu/fmriprep/'
# output_dir = '/output/' 

#For Daniel's computer
input_dir = 'data/CNP_clinical/func/'
output_dir = 'data/CNP_clinical/func/'


n_jobs = 20 #amount of cores
cv=4

parcellation = '197_'
description = 'CNP_clinical_func_validation_age_'
log_file = parcellation+description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [3]:
# Load healthy control data
input_dir = '/Users/danielmlow/Dropbox/neurohack/brain_age/data/CNP_func/'
loaded = np.load(input_dir+'CNP_func_gridsearch_age_18-08-09-22-05-36CNP_func'+parcellation+'train_set.npz')
X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']

# Load clinical population
input_dir = '/Users/danielmlow/Dropbox/neurohack/brain_age/data/CNP_clinical/func/'
disorder = 'adhd_'
# the clinical group was split 50-50 in validation (called "train") and test set (called "test")
loaded = np.load(input_dir+'CNP_func_gridsearch_age_18-08-10-02-50-17CNP_func'+disorder+parcellation+'train_set.npz')
X_validation_adhd, y_validation_adhd, random_subj_train_adhd = loaded['a'], loaded['b'], loaded['c']

disorder = 'bipolar_'
# the clinical group was split 50-50 in validation (called "train") and test set (called "test")
loaded = np.load(input_dir+'CNP_func_gridsearch_age_18-08-10-02-50-17CNP_func'+disorder+parcellation+'train_set.npz')
X_validation_bipolar, y_validation_bipolar, random_subj_train_bipolar = loaded['a'], loaded['b'], loaded['c']

disorder = 'schz_'
# the clinical group was split 50-50 in validation (called "train") and test set (called "test")
loaded = np.load(input_dir+'CNP_func_gridsearch_age_18-08-10-02-50-17CNP_func'+disorder+parcellation+'train_set.npz')
X_validation_schz, y_validation_schz, random_subj_train_schz = loaded['a'], loaded['b'], loaded['c']



In [4]:
print(y_train.shape)
print(X_train.shape)

def reshape_3_to_2D(X):
    return np.reshape(X, (X.shape[0], X.shape[2]))
    
# X_train = reshape_3_to_2D(X_train)

X_train = reshape_3_to_2D(X_train)
X_validation_adhd = reshape_3_to_2D(X_validation_adhd)
X_validation_bipolar = reshape_3_to_2D(X_validation_bipolar)
X_validation_schz = reshape_3_to_2D(X_validation_schz)




print(X_train.shape)

(84,)
(84, 1, 19306)
(84, 19306)


In [5]:
X_train.shape

(84, 19306)

In [6]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.30, random_state=42)

## Validation 

Here we take the validation set of the clinical population and "diagnose" their brain age. 


In [7]:
from sklearn.pipeline import Pipeline

if toy:
    X_train = X_train[:8]
    y_train = y_train[:8]
    X_validation = X_validation[:8]
    y_validation = y_validation[:8]


# Remove features with too low between-subject variance
# Here we use a classical univariate feature selection based on F-test,
# namely Anova.
# We have our predictor (SVR), our feature selection (SelectKBest), and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
               
anova_svr = Pipeline([('svr', LinearSVR(C=1.))])

# parameters = [{'anova__k': [2000],
#                'svr__C': [0.1,1]}]


anova_svr.fit(X_train, y_train)

# clf = SVR(kernel='linear', C=0.01)
# clf.fit(X_train, y_train)
# predictions = clf.predict(X_validation)




# r2 = r2_score(y_validation, predictions)
# mean_scores = np.array(grid.cv_results_['neg_mean_absolute_error'])
# mean_scores = mean_scores.reshape(len(C_OPTIONS), -1, len(N_FEATURES_OPTIONS))
# # select score for best C
# mean_scores = mean_scores.max(axis=0)
# bar_offsets = (np.arange(len(N_FEATURES_OPTIONS)) *
#                (len(reducer_labels) + 1) + .5)







Pipeline(memory=None,
     steps=[('svr', LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0))])

(25, 19306)

In [8]:
def mean_error(predictions, true_ages):
    errors = []
    for pred, true in zip(predictions, true_ages):
        errors.append(pred-true)
    return np.mean(errors), np.std(errors)
    

'''
pred true
40    45   -5 brain age is five years younger that true age
30    25   5
'''


predictions_healthy = anova_svr.predict(X_validation)

predictions_adhd = anova_svr.predict(X_validation_adhd)


predictions_bipolar = anova_svr.predict(X_validation_bipolar)


predictions_schz = anova_svr.predict(X_validation_schz)




mean_error_healthy, std_healthy = mean_error(predictions_healthy, y_validation)
mean_error_adhd, std_adhd = mean_error(predictions_adhd, y_validation_adhd)
mean_error_bipolar, std_bipolar = mean_error(predictions_adhd, y_validation_bipolar)
mean_error_schz, std_schz = mean_error(predictions_adhd, y_validation_schz)
print('healthy: ', mean_error_healthy.round(2),std_healthy.round(2))
print('adhd: ', mean_error_adhd, std_adhd.round(2))
print('bipolar: ', mean_error_bipolar, std_bipolar.round(2))
print('schz: ', mean_error_schz, std_schz.round(2), std_schz.round(2))

print('mean age:')
print(np.mean(y_train))
print(np.mean(y_validation_adhd))
print(np.mean(y_validation_bipolar))
print(np.mean(y_validation_schz))

healthy:  -0.06 8.26
adhd:  -2.133288553392293 9.15
bipolar:  -4.183288553392294 10.12
schz:  -8.733288553392292 8.96 8.96
mean age:
31.32758620689655
32.4
35.375
38.64


In [9]:

# Diagnose brain age


print('adhd: ', mean_error_adhd)
print('bipolar: ', mean_error_bipolar)
print('schz: ', mean_error_schz)


adhd:  -2.133288553392293
bipolar:  -4.183288553392294
schz:  -8.733288553392292


In [10]:


with open(output_dir+'log_'+log_file+'.txt', '+a') as f:
    f.write(str('healthy: ')+str( mean_error_healthy.round(2),std_healthy.round(2)))
    f.write(str('adhd: ')+str(mean_error_adhd, std_adhd.round(2)))
    f.write('bipolar: '+str(mean_error_bipolar, std_bipolar.round(2)))
    f.write(str('schz: ')+str( mean_error_schz, std_schz.round(2), std_schz.round(2)))
    f.write(str('mean age:'))
    f.write(str(np.mean(y_train)))
    f.write(str(np.mean(y_validation_adhd)))
    f.write(str(np.mean(y_validation_bipolar)))
    f.write(str(np.mean(y_validation_schz)))

TypeError: str() argument 2 must be str, not numpy.float64

In [ ]:
output_dir+'log_'+log_file+'.txt'

In [ ]:
# age_pred = grid.predict(gm_maps_masked)
# np.mean(np.abs(age_pred-y_age))
# y_age

In [ ]:
# r2 score